# Explainable AI

This notebook summarizes relevant information regarding some state-of-the-art algorithms used to explain ML models, including:

- <a href='#ebm'>Explainable Boosting Machine (EBM)</a>
- <a href='#lime'>Local Interpretable Model-Agnostic Explanations (LIME)</a>
- <a href='#shap'>SHapley Additive exPlanations (SHAP)</a>
- <a href='#pdm'>Partial dependence model</a>
- <a href='#morris'>Morris Sensitivity</a>

<a id='ebm'></a>

## Explainable Boosting Machine (EBM)


- Developed at Microsoft Research


- Designed to have accuracy comparable to state-of-the-art methods like Random Forest and Boosted Trees, while being highly intelligibile and explainable


- EBM is a generalized additive model (GAM) with a few major improvements over traditional GAMs:
    - Standard GAMs usually model the dependent variable as a sum of univariate models; their accuracy is significantly less than more complex models that permit interactions between variables
    - EBM can automatically detect and include pairwise interaction terms 
    - EBM learns each feature function using bagging and gradient boosting
    - Because EBM is an additive model, each feature contributes to predictions in a modular way that makes it easy to reason about the contribution of each feature to the prediction


Algorithm explanation: https://www.youtube.com/watch?v=MREiHgHgl0k&ab_channel=MicrosoftDeveloper  
EBM is a fast implementation of the GA2M algorithm: https://dl.acm.org/doi/10.1145/2487575.2487579


<a id='lime'></a>

## Local Interpretable Model-Agnostic Explanations (LIME)


In order to figure out what parts of the input are contributing to the prediction, LIME tests what happens to the predictions when you give variations of your data to your machine learning model. 


**How LIME works:**

1. You train a black-box model and select the instance for which you want an explanation
2. LIME Generates a new dataset consisting of permuted samples from your data and the corresponding predictions from your black-box model as targets
3. Perturbs your dataset (adds noise to continuous features, removes words in text, hides parts of images)
4. Trains an interpretable model on the new dataset, weighting the perturbed data points by their proximity to the original example
5. Explains the prediction by interpreting this new model


<img src="images/lime1.png" width="600" height="600" align="center"/>
<br clear="all" />

<font color='gray'>Image source: [4]</font>


<img src="images/lime2.png" width="800" height="800" align="center"/>
<br clear="all" />

<font color='gray'>Image source: [5]</font>


**Problems:**
1. Instability of the explanations:
    - Explanations of similar examples might be totally different
    - If you repeat the sampling process, then the explanations that come out can be different
2. LIME suffers from labels and data shift (training and test distributions are different)
    - Instances generated by LIME’s perturbation procedure are significantly different from training instances drawn from the underlying distribution
3. Explanation depends on the choice of LIME hyperparameters


**Note:**
The explanations created with local surrogate models can use other (interpretable) features than the original model was trained on (e.g. a regression model could be trained on components of a principal component analysis (PCA), but LIME might be trained on the original data)


<a id='shap'></a>

## SHapley Additive exPlanations (SHAP)


SHAP can explain any machine learning model. It is based on Shapley values, a concept coming from game theory. It's goal is to explain the prediction of an instance by computing the contribution of each feature to the prediction. 

For example, imagine that we have 3 features in a model to predict income: Age, Gender, and Job. Shapley quantifies the contribution that each feature brings to the model. It is based on the idea that the outcome of each possible combination of features should be used to get the importance of one single feature. So, to calculate the contribution of feature Age, we need to consider all the interactions of Age with the other features and their collective contribution as well.

The SHAP value of Age will be the (weighted) average of the marginal contributions of this feature. A marginal contribution of a feature is the difference between predictions by including the feature and without including the feature. 

<img src="images/shap2.png" width="500" height="500" align="center"/>
<br clear="all" />

<font color='gray'>Image source: [10]</font>

The SHAP values can show how much each predictor contributes, either positively or negatively, to the target variable. Summing the SHAP values of each feature of a given observation yields the difference between the prediction of the model and the average prediction for the dataset.

<img src="images/shap1.png" width="800" height="800" align="center"/>
<br clear="all" />

<font color='gray'>Image source: [11]</font>

Considering every feature and their interactions, if F is the total number of features, SHAP requires to train 2^F distinct predictive models. As F increases, this becames inapplicable, so SHAP libraries employ approximations and samplings that make this feasible.

<a id='pdm'></a>

## Partial dependence model

- A partial dependence plot shows the global effect of a feature on the outcome of a ML model. 

- The partial dependence function for a feature at a value represents the average prediction if we have all data points assume that feature value.

<img src="images/pdp.png" width="600" height="600" align="center"/>
<br clear="all" />

<font color='gray'>Image source: [11]</font>

<a id='morris'></a>

## Morris Sensitivity

 - Morris method is a statistical method for global sensitivity analysis (SA)
 - It is a one-step-at-a-time method (OAT), meaning that in each run only one input parameter is given a new value 
 - In OAT SA design, all input variables in question are changed by the same relative amount. However, the Morris method takes into account changing only one variable between a pair of model simulations
 - Identification and ranking of the important variables are done on the basis of the difference computed between a pair of model simulations

- The input space is considered as a grid. The algorithm starts at a randomly chosen point in the K-dimensional space and creates a trajectory through all the K-dimensional variable space.

<img src="images/morris1.png" width="600" height="600" align="center"/>
<br clear="all" />

<font color='gray'>Image source: [14]</font>

- This method captures output variation when one of the trajectory points is moved to one of its closest neighbors. This variation is called an elementary effect:

<img src="images/morris2.png" width="600" height="600" align="center"/> 
<br clear="all" />
  
*i = 1,..., R (R trajectories)  
j = 1,..., K (K dimensions)*  

- A certain number of trajectories R are generated, in order to observe the consequence of elementary effects anywhere in the input space.

- Finally, these elementary effects are used to estimate global sensitivity with the following indicators: mu, mu_star, sigma.

<br clear="all" />

**Summary statistics on simulated elementary effects:**

**Mu**:
- The mean assesses the overall impact (influence) of the feature on the model prediction
    - A large mean indicates large changes in the response when perturbing a feature (an influential parameter)
- Note that it can aggregate very different strengths of effects and cancel opposite effects
    
**Mu_star**:

<img src="images/morris3.png" width="150" height="150" align="center"/>
<br clear="all" />

- A more robust indicator, it is computed as the mean of the absolute value of the elementary effects
    
    
**Sigma**:

<img src="images/morris2.png" width="600" height="600" align="center"/>
<br clear="all" />

- The standard deviation or variance is a measure on non-linear and/or interaction effects:
    - A large value indicates that the influence of the feature depends highly on the interaction with other features
    - A small value indicates similar influences for different sample points, implying that the effect of the feature is independent of its interaction with other features
        
   
        
<font color='gray'>Equations source: [14]</font>

## Resources

EBM  
[1] https://medium.com/analytics-vidhya/model-interpretation-with-microsofts-interpret-ml-85aa0ad697ae  
[2] https://arxiv.org/pdf/1909.09223.pdf

LIME  
[3] https://homes.cs.washington.edu/~marcotcr/blog/lime/  
[4] https://christophm.github.io/interpretable-ml-book/lime  
[5] https://www.oreilly.com/content/introduction-to-local-interpretable-model-agnostic-explanations-lime/  
[6] https://towardsdatascience.com/understanding-model-predictions-with-lime-a582fdff3a3b  
[7] https://towardsdatascience.com/whats-wrong-with-lime-86b335f34612  

SHAP  
[8] https://christophm.github.io/interpretable-ml-book/shap.html  
[9] https://medium.com/@prarthana1/unboxing-machine-learning-models-with-shap-for-better-interpretation-5b2caa81e3da   
[10] https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30  
[11] https://towardsdatascience.com/three-model-explanability-methods-every-data-scientist-should-know-c332bdfd8df  

Partially dependence model  
[11]

Morris sensitivity  
[12] https://www.youtube.com/watch?v=P8Rfipkid3w&ab_channel=JefCaers  
[13] https://openmole.org/Sensitivity.html  
[14] https://link.springer.com/article/10.1007/s11600-019-00356-5   
